In [2]:
#библиотеки, необходимые для формирования запросов сайту и получение ответов
#и поиска в полученной с сайта информации необхоимых для проекта данных
from bs4 import BeautifulSoup
import requests as req
import lxml.html as html
import urllib3

import time #библиотека для установки паузы между запросами к сайту

#общие библиотеки
import pandas as pd
import numpy as np

import re

In [2]:
#формируем header для включения его в запросы, так как сайты могут блокировать запросы без него
header = {'user-agent': 'Chrome/108.0.0.0'}
urllib3.disable_warnings()

In [8]:
url_star = 'https://www.booking.com/searchresults.en-us.html?label=gen173nr-1FCAEoggI46AdIM1gEaMIBiAEBmAExuAEXyAEM2AEB6AEB-AECiAIBqAIDuALlmsecBsACAdICJDFmMDhmM2QxLTMxYTAtNDMzMC05ODYwLTllNWU2M2EyMDZiN9gCBeACAQ&aid=304142&ss=Barcelona&ssne=London&ssne_untouched=London&lang=en-us&src=index&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&nflt=ht_id%3D204%3Bclass%3D'
url_all_star = []

for i in range(2):
    url_all_star.append(url_star + str(i+4) + '&order=distance_from_search')

In [9]:
urls_hotel_4 = []
urls_hotel_5 = []

for star in range(6):
    time.sleep(3)  
    if star == 0:
        
        for i in range(0, 201, 25):
            try:
                response = req.get(url_all_star[star] + '&offset=' + str(i), headers=header, timeout=50, verify=False)
                page = BeautifulSoup(response.text,'html.parser')
            except:
                print('Ошибка запроса')
                
                
            for a in page.find_all('a', class_='e13098a59f'):
                urls_hotel_4.append(a['href'])
        time.sleep(3) 
    if star == 1:
        for i in range(0, 26, 25):
            try:
                response = req.get(url_all_star[star] + '&offset=' + str(i), headers=header, timeout=50, verify=False)
                page = BeautifulSoup(response.text,'html.parser')
            except:
                print('Ошибка запроса')
                
                
            for a in page.find_all('a', class_='e13098a59f'):
                urls_hotel_5.append(a['href'])
        time.sleep(3)
    


print(len(urls_hotel_4))
print(len(urls_hotel_5))

206
43


In [10]:
urls_hotel_4_en = []
urls_hotel_5_en = []

    
for i in range(len(urls_hotel_4)):
    parts = urls_hotel_4[i].split('hapos=' + str(i+1))
    urls_hotel_4_en.append(parts[0] + ('hapos=' + str(i+1)) + '&lang=en-us' + parts[1])
    
for i in range(len(urls_hotel_5)):
    parts = urls_hotel_5[i].split('hapos=' + str(i+1))
    urls_hotel_5_en.append(parts[0] + ('hapos=' + str(i+1)) + '&lang=en-us' + parts[1])


print(len(urls_hotel_4_en))
print(len(urls_hotel_5_en))

206
43


In [11]:
data_array_full = np.empty(4, 'str')

tags = ['span', 'div', 'div']
classes = ['hp_address_subtitle js-hp_address_subtitle jq_tooltip', 'd8eab2cf7f c90c0a70d3 db63693c62',
           'b5cd09854e f0d4d6a2f5 e46e88563a']

all_star_url = [urls_hotel_4_en, urls_hotel_5_en]


count_star_0 = 3

for urls in all_star_url:
    count_star_0 += 1
    for url in urls:
        try:
            response = req.get(url, headers=header, timeout=50, verify=False)
            page = BeautifulSoup(response.text,'html.parser')
        except:
            print('ошибка запроса')
        
    
        data = []
        data.append(count_star_0)
        for i in range(3):
            try:
                elem = page.find_all(tags[i], classes[i])
                elem = elem[0].text
            except:
                elem = np.NaN
        
                
            data.append(elem)
            
        data_array = np.asarray(data).reshape(1,4)
        data_array_full = np.row_stack((data_array_full,data_array))
    
        time.sleep(3)

ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса
ошибка запроса


In [16]:
columns = np.array(['hotel_star', 'hotel_address','reviews_count','quality'])
hotel_barcelona = pd.DataFrame(data=data_array_full, columns=columns)
hotel_barcelona.replace('nan', np.NaN, inplace=True)
hotel_barcelona = hotel_barcelona.dropna(thresh=2).reset_index(drop=True)
hotel_barcelona

,hotel_star,hotel_address,reviews_count,quality
0,,,,
1,4,"\nPlaza de Cataluña,10, Eixample, 08002 Barcel...","2,287 reviews",Excellent
2,4,"\nPlaza de Catalunya, 19, Ciutat Vella, 08002 ...","2,135 reviews",Wonderful
3,4,"\nBergara, 11, Eixample, 08002 Barcelona, Spain\n","2,826 reviews",Very Good
4,4,"\nBergara, 8, Eixample, 08002 Barcelona, Spain\n","1,288 reviews",Excellent
...,...,...,...,...
245,5,"\nDoctor Carulla, 25, Sarrià-St. Gervasi, 0801...",253 reviews,Wonderful
246,5,"\nAvenida Tibidabo, 1, Sarrià-St. Gervasi, 080...",208 reviews,Wonderful
247,5,"\nPlaca de Pius XII, 4, Les Corts, 08028 Barce...","1,773 reviews",Excellent
248,5,"\nAvinguda del Tibidabo 32, Sarrià-St. Gervasi...",572 reviews,Wonderful


,Unnamed: 0,hotel_star,hotel_address,reviews_count,quality
0,0,NaN,NaN,NaN,NaN
1,1,4.0,plazadecatalua10eixample08002barcelonaspain,"2,287 reviews",Excellent
2,2,4.0,plazadecatalunya19ciutatvella08002barcelonaspain,"2,135 reviews",Wonderful
3,3,4.0,bergara11eixample08002barcelonaspain,"2,826 reviews",Very Good
4,4,4.0,bergara8eixample08002barcelonaspain,"1,288 reviews",Excellent
...,...,...,...,...,...
231,245,5.0,doctorcarulla25sarristgervasi08017barcelonaspain,253 reviews,Wonderful
232,246,5.0,avenidatibidabo1sarristgervasi08022barcelonaspain,208 reviews,Wonderful
233,247,5.0,placadepiusxii4lescorts08028barcelonaspain,"1,773 reviews",Excellent
234,248,5.0,avingudadeltibidabo32sarristgervasi08022barcel...,572 reviews,Wonderful


In [4]:
hotel_df = pd.read_csv('C:/Users/Dmitriy/Desktop/SkillFactory/project_3/hotels.csv', sep=',')
hotel_df.shape

(386803, 57)

In [6]:
hotel_barcelona['hotel_address'] = hotel_barcelona['hotel_address'].apply(
    lambda x: re.sub("[^0-9a-zA-Z]"," ",x).strip().lower().replace(' ', ''))

In [18]:
hotel_barcelona['hotel_address']

0                                                       
1            plazadecatalua10eixample08002barcelonaspain
2       plazadecatalunya19ciutatvella08002barcelonaspain
3                   bergara11eixample08002barcelonaspain
4                    bergara8eixample08002barcelonaspain
                             ...                        
245     doctorcarulla25sarristgervasi08017barcelonaspain
246    avenidatibidabo1sarristgervasi08022barcelonaspain
247           placadepiusxii4lescorts08028barcelonaspain
248    avingudadeltibidabo32sarristgervasi08022barcel...
249    carreteravallvidreratibidabo8393hortaguinard08...
Name: hotel_address, Length: 250, dtype: object

In [10]:
hotel_df['hotel_address'] = hotel_df['hotel_address'].apply(
    lambda x: x.lower().replace(' ', ''))

In [11]:
hotel_df['hotel_address']

0         strattonstreetmayfairwestminsterboroughlondonw...
1         130134southamptonrowcamdenlondonwc1b5afunitedk...
2                   151bisruederennes6tharr75006parisfrance
3                216avenuejeanjaures19tharr75019parisfrance
4                      molenwerf11014agamsterdamnetherlands
                                ...                        
386798    9knaresboroughplacekensingtonandchelsealondons...
386799    landstraerhauptstrae15503landstrae1030viennaau...
386800      2931gowerstreetcamdenlondonwc1e6hgunitedkingdom
386801    31greatcumberlandplacewestminsterboroughlondon...
386802    25courtfieldgardenskensingtonandchelsealondons...
Name: hotel_address, Length: 386803, dtype: object

In [12]:
hotel_df = hotel_df.merge(
                hotel_barcelona,
                on='hotel_address',
                how='left')

In [17]:
null = hotel_df[hotel_df['city_hotel'] == 'Barcelona'].isnull().mean() * 100
null_per = null[null > 0]
null_per

hotel_star       9.489941
reviews_count    9.489941
quality          9.489941
dtype: float64

In [8]:
dupl_columns = list(hotel_barcelona.columns) 

mask = hotel_barcelona.duplicated(subset=dupl_columns) 
hotel_duplicates = hotel_barcelona[mask]
print(f'Число найденных дубликатов: {hotel_duplicates.shape[0]}')

Число найденных дубликатов: 0


In [9]:
hotel_barcelona = hotel_barcelona.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {hotel_barcelona.shape[0]}')

Результирующее число записей: 236


In [21]:
hotel_barcelona.to_csv('C:/Users/Dmitriy/Desktop/SkillFactory/project_3/parsing_hotels/hotel_barcelona.csv')